- Top 10 Paths mit anzahl versptungen (balkendiagramm)<br>
- Top 10 Bahnhöfe mit anzahl verspätungen geteilt durch anzahl fahrten (also quasi Prozentsatz der verspätungen am Bahnhof)(balkendiagramm)<br>
- Beides nochmal mit durchschnittlichen Verspätungslängen like 1

In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import date, time, datetime

In [2]:
#Create dataframe from CSV
df = pd.read_csv("../data/fromAPI/cleaned_hourly_all.csv")

In [10]:
df.path.nunique

<bound method IndexOpsMixin.nunique of 0            Stolberg(Rheinl)Hbf|Eilendorf|Aachen-Rothe Erde
1          Stolberg(Rheinl)Hbf Gl.44|Eschweiler-St.Jöris|...
2                                                        NaN
3                                                        NaN
4                                                        NaN
                                 ...                        
1370584    Bergisch Gladbach|Duckterath|Köln-Dellbrück|Kö...
1370585                                                  NaN
1370586    Bergisch Gladbach|Duckterath|Köln-Dellbrück|Kö...
1370587                                                  NaN
1370588                                                  NaN
Name: path, Length: 1370589, dtype: object>

In [ ]:
df.groupby("path")